<a href="https://colab.research.google.com/github/hang-1n-there/Resnet_CIFA10/blob/main/resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class BasicBlock(nn.Module):
  mul = 1 # 출력 채널 수를 조절

  def __init__(self, in_channel, out_channel, stride=1):
    super(BasicBlock, self).__init__()

    self.conv1 = nn.Conv2d(in_channel, out_channel, kernel_size=3, stride=stride, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(out_channel)

    self.conv2 = nn.Conv2d(in_channel, out_channel, kernel_size=3, stride=stride, padding=1, bias=False)
    self.bn2 = nn.BatchNorm2d(out_channel)

    self.shortcut = nn.Sequential()

    if stride !=1:
      self.shortcut = nn.Sequential(
        nn.Conv2d(in_channel, out_channel, kernel_size=3, stride=stride, padding=1, bias=False),
        nn.BatchNorm2d(out_channel),
      )

  def forward(self,x):
    result = self.conv1(x)
    result = self.bn1(result)
    result = F.relu(result)
    result = self.conv2(x)
    result = self.bn2(result)
    result += self.shortcut(x)
    result = F.relu(result)

    return result

class BottleNeck(nn.Module):
    mul = 4

    def __init__(self, in_channel, out_channel, stride=1):
        super(BottleNeck, self).__init__()

        self.conv1 = nn.Conv2d(in_channel, out_channel, kernel_size=1, stride=stride, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channel)

        self.conv2 = nn.Conv2d(out_channel, out_channel, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channel)

        self.conv3 = nn.Conv2d(out_channel, out_channel * self.mul, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channel * self.mul)

        self.shortcut = nn.Sequential()

        if stride != 1 or in_channel != out_channel * self.mul:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channel, out_channel * self.mul, kernel_size=1, stride=stride, padding=0, bias=False),
                nn.BatchNorm2d(out_channel * self.mul)
            )

    def forward(self, x):
        result = F.relu(self.bn1(self.conv1(x)))
        result = F.relu(self.bn2(self.conv2(result)))
        result = self.bn3(self.conv3(result))

        shortcut = self.shortcut(x)
        result += shortcut
        result = F.relu(result)

        return result

class Resnet(nn.Module):
  def __init__(self, block, num_blocks, num_classes = 10):
    super(Resnet, self).__init__()

    self.in_channel = 64

    self.conv1 = nn.Conv2d(3, self.in_channel, kernel_size=7, stride=2, padding=3)
    self.bn1 = nn.BatchNorm2d(self.in_channel)
    self.maxpool1 = nn.MaxPool2d(kernel_size=3,stride=2,padding=1)

    self.layer1 = self.make_layer(block, 64, num_blocks[0], stride=1)
    self.layer2 = self.make_layer(block, 128, num_blocks[0], stride=1)
    self.layer3 = self.make_layer(block, 256, num_blocks[0], stride=1)
    self.layer4 = self.make_layer(block, 518, num_blocks[0], stride=1)
    self.avgpool = nn.AdaptiveAvgPool2d((1,1))
    self.linear = nn.Linear(2072, num_classes)

  def make_layer(self, block, out_channel, num_blocks, stride):
    # 다운샘플링을 위해 첫 번째 블럭에만 stride 적용
    strides = [stride] + [1] * (num_blocks - 1)
    layers = []

    for num_block in range(num_blocks):
      layers.append(block(self.in_channel, out_channel, strides[num_block]))
      self.in_channel = block.mul * out_channel

    return nn.Sequential(*layers)

  def forward(self, x):
    result = self.conv1(x)
    result = self.bn1(result)
    result = F.relu(result)
    result = self.maxpool1(result)
    result = self.layer1(result)
    result = self.layer2(result)
    result = self.layer3(result)
    result = self.layer4(result)
    result = self.avgpool(result)
    result = torch.flatten(result, 1)
    result = self.linear(result)

    return result

def ResNet18():
    return Resnet(BasicBlock, [2, 2, 2, 2])

def ResNet34():
    return Resnet(BasicBlock, [3, 4, 6, 3])

def ResNet50():
    return Resnet(BottleNeck, [3, 4, 6, 3])

def ResNet101():
    return Resnet(BottleNeck, [3, 4, 23, 3])

def ResNet152():
    return Resnet(BottleNeck, [3, 8, 36, 3])

In [ ]:
import torchvision
import torch.optim as optim
import torchvision.transforms as transforms
import os
import torchvision.models as models

In [ ]:
# Simple Learning Rate Scheduler
def lr_scheduler(optimizer, epoch):
    lr = learning_rate
    if epoch >= 50:
        lr /= 10
    if epoch >= 100:
        lr /= 10
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

# Xavier
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=8)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=8)

100%|██████████| 170498071/170498071 [00:04<00:00, 40829185.89it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
device = 'cuda'
model = ResNet50()

In [ ]:
model.apply(init_weights)
model = model.to(device)

<ipython-input-4-cb3abaf3ef20>:14: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight)


In [ ]:
learning_rate = 0.1
num_epoch = 150
model_name = 'model.pth'

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0001)

train_loss = 0
valid_loss = 0
correct = 0
total_cnt = 0
best_acc = 0

In [ ]:
# Train
from tqdm import tqdm

for epoch in tqdm(range(num_epoch)):
    print(f"====== { epoch+1} epoch of { num_epoch } ======")
    model.train()
    lr_scheduler(optimizer, epoch)
    train_loss = 0
    valid_loss = 0
    correct = 0
    total_cnt = 0

    for step, batch in enumerate(train_loader):
        #  input and target
        batch[0], batch[1] = batch[0].to(device), batch[1].to(device)
        optimizer.zero_grad()

        logits = model(batch[0])
        loss = loss_fn(logits, batch[1])
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predict = logits.max(1)

        total_cnt += batch[1].size(0)
        correct +=  predict.eq(batch[1]).sum().item()

        if step % 100 == 0 and step != 0:
            print(f"\n====== { step } Step of { len(train_loader) } ======")
            print(f"Train Acc : { correct / total_cnt }")
            print(f"Train Loss : { loss.item() / batch[1].size(0) }")

    correct = 0
    total_cnt = 0

    # Test
    with torch.no_grad():
        model.eval()
        for step, batch in enumerate(test_loader):
            # input and target
            batch[0], batch[1] = batch[0].to(device), batch[1].to(device)
            total_cnt += batch[1].size(0)
            logits = model(batch[0])
            valid_loss += loss_fn(logits, batch[1])
            _, predict = logits.max(1)
            correct += predict.eq(batch[1]).sum().item()
        valid_acc = correct / total_cnt
        print(f"\nValid Acc : { valid_acc }")
        print(f"Valid Loss : { valid_loss / total_cnt }")

        if(valid_acc > best_acc):
            best_acc = valid_acc
            torch.save(model, model_name)
            print("Model Saved!")

  0%|          | 0/150 [00:00<?, ?it/s]

====== 1 epoch of 150 ======


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()



====== 100 Step of 196 ======
Train Acc : 0.15485767326732675
Train Loss : 0.008431224152445793


  1%|          | 1/150 [01:50<4:34:43, 110.63s/it]

...

Model Saved!
====== 22 epoch of 150 ======

====== 100 Step of 196 ======
Train Acc : 0.7583152846534653
Train Loss : 0.002741477685049176

Valid Acc : 0.7418
Valid Loss : 0.0029967024456709623


 15%|█▍        | 22/150 [43:55<4:18:05, 120.98s/it]

Model Saved!
====== 23 epoch of 150 ======

====== 100 Step of 196 ======
Train Acc : 0.7662824876237624
Train Loss : 0.002762447576969862


 15%|█▍        | 22/150 [45:35<4:25:14, 124.33s/it]


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,8))
plt.plot(num_epoch, train_loss, num_epoch, valid_loss)
plt.legend(labels=['train', 'valid'], loc='upper left')
plt.grid(True)
plt.show()